In [78]:
# load library
from datetime import datetime
import math
import numpy as np
import pandas as pd

In [79]:
# load data
csv_file = "Data/Episodes.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)
csv_df = pd.DataFrame(csv_data)
Episodes_df1 = csv_df[["PID","EpisodeID"]]


#load data - Episodes_new is just the original excel file "Data/Episodes" changed to a csv file for better formatting.
csv_file = "Data/Episodes_new.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)
csv_df = pd.DataFrame(csv_data)
Episodes_df2 = csv_df[["EpisodeID","ContractType","AdmitDate","DischargeDate"]]

In [80]:
Episodes_df = pd.merge(Episodes_df1, Episodes_df2, how='left', on="EpisodeID")

In [81]:
Episodes_df

,PID,EpisodeID,ContractType,AdmitDate,DischargeDate
0,530,1,CT1,12/1/2019,1/30/20
1,1107,3,CT2,12/1/2019,12/7/19
2,1107,4,CT2,12/1/2019,12/2/19
3,842,6,CT2,12/1/2019,12/4/19
4,1178,7,CT2,12/1/2019,12/17/19
...,...,...,...,...,...
39840,1771,56813,CT1,1/1/2019,1/15/19
39841,1177,56814,CT1,1/1/2019,1/14/19
39842,1187,56815,CT1,1/1/2019,2/7/19
39843,1771,56816,CT1,1/1/2019,1/18/19


In [82]:
# get month information from AdmitDate
Admitmonth_list = []
for date in Episodes_df["AdmitDate"]:
    Admitmonth_list.append(int(date.split('/')[0]))
    
# get month information from DischargeDate
Dischargemonth_list = []
for date in Episodes_df["DischargeDate"]:
    if str(date) == "nan":
        Dischargemonth_list.append(None)
    elif str(date[-1]) != "9":
        Dischargemonth_list.append(12)
    else:
        Dischargemonth_list.append(int(str(date).split('/')[0]))
        
# deal with none in dischargedate     
new_column = []
for i in range(len(Dischargemonth_list)):
    if Dischargemonth_list[i] == None:
        new_column.append('#')
        continue
    new_column.append((Admitmonth_list[i],Dischargemonth_list[i]+1))
Episodes_df['month_range'] = new_column

In [83]:
#create empty dict and keep the infomation into dictionay for further operation
_dict = dict()
for i in range(len(Episodes_df["PID"])):
    if Episodes_df['month_range'][i] == '#':
        continue
    if Episodes_df["PID"][i] not in _dict:
        _dict[Episodes_df["PID"][i]] = list(range(Episodes_df['month_range'][i][0],Episodes_df['month_range'][i][1]))
    else:
        _dict[Episodes_df["PID"][i]].extend(list(range(Episodes_df['month_range'][i][0],Episodes_df['month_range'][i][1])))

In [84]:
# sort pid order from small to large
_dict = sorted(_dict.items(), key = lambda x:x[0])
len(_dict)

1231

In [85]:
# Create a two-dimensional array for keeping information in next operation
data=np.random.randn(1231,13)
data

array([[ 0.17671777,  1.92574889, -1.73394625, ...,  0.53150915,
        -0.07133019, -0.20124385],
       [ 0.98719251,  0.57991447,  0.51649972, ...,  0.562043  ,
        -0.8720392 , -1.71604434],
       [ 0.50075495, -1.14473537, -0.91838864, ..., -1.32701118,
         0.45156289, -0.19697758],
       ...,
       [-2.10954412,  0.74727437,  1.5579319 , ..., -0.47860891,
         0.16666261, -1.51910234],
       [ 0.08119004,  0.47716554, -0.40948251, ...,  1.6907481 ,
        -0.41277337,  0.42923637],
       [-0.06136129, -1.60116385,  0.26359247, ...,  0.48926118,
        -0.39317862, -0.7391276 ]])

In [86]:
# create a dataframe based on a two-dimensional array
df_ = pd.DataFrame(data,columns=['PID', 'M1', 'M2', 'M3','M4','M5','M6','M7','M8','M9','M10','M11','M12'],index=list(range(0,1231)),dtype=np.int64)
df_

,PID,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
0,0,1,-1,1,-1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,-1,-2,1,1,0,0,0,-1
2,0,-1,0,0,0,0,0,0,1,0,-1,0,0
3,1,0,0,-1,1,1,0,0,1,-1,0,0,-1
4,0,0,-1,0,0,-1,1,0,1,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,1,0,2,0,0,0,1,0,1,1,-1,1,0
1227,1,1,1,0,0,1,0,0,0,0,0,0,0
1228,-2,0,1,0,-1,0,-1,1,0,0,0,0,-1
1229,0,0,0,0,0,0,-1,0,1,0,1,0,0


In [87]:
# calculate the number of patient in each snf every month
# and keep the information into the dataframe we created
for i in range(len(_dict)):
    df_["PID"][i] = int(_dict[i][0])
    df_['M1'][i] = int(_dict[i][1].count(1))
    df_['M2'][i] = int(_dict[i][1].count(2))
    df_['M3'][i] = int(_dict[i][1].count(3))
    df_['M4'][i] = int(_dict[i][1].count(4))
    df_['M5'][i] = int(_dict[i][1].count(5))
    df_['M6'][i] = int(_dict[i][1].count(6))
    df_['M7'][i] = int(_dict[i][1].count(7))
    df_['M8'][i] = int(_dict[i][1].count(8))
    df_['M9'][i] = int(_dict[i][1].count(9))
    df_['M10'][i] = int(_dict[i][1].count(10))
    df_['M11'][i] = int(_dict[i][1].count(11))
    df_['M12'][i] = int(_dict[i][1].count(12))
df_

,PID,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
0,6,11,5,7,6,4,3,3,5,6,5,3,11
1,8,20,12,14,12,16,17,15,8,6,6,4,7
2,9,10,10,15,12,11,11,12,7,10,6,10,6
3,10,2,0,1,0,1,0,0,0,0,0,0,0
4,11,4,6,5,6,3,4,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,1808,4,2,4,1,2,7,2,5,5,3,6,4
1227,1810,0,0,0,0,0,2,0,0,0,0,0,0
1228,1811,0,1,1,0,0,1,1,0,0,1,1,0
1229,1812,0,3,0,1,0,0,1,0,0,0,0,0


In [88]:
# load dataset including county information and save it into dataframe
county_complete = pd.read_csv('Data/county_complete.csv')  
county_complete

,ProviderName,PID,ProviderPracticeAddress1,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode,County,zip,lat,...,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,648 Medical Squadron,1,8006 CHENNAULT RD,STE 1,BROOKS CITY BASE,TX,78235,Bexar,78235.0,29.34604,...,1176.0,131.5,48029.0,Bexar,{'48029':100},Bexar,48029,False,False,America/Chicago
1,Abilene Nursing and Rehabilitation Center,2,2630 OLD ANSON RD,NaN,ABILENE,TX,79603,Taylor,79603.0,32.40153,...,23064.0,160.5,48441.0,Taylor,{'48441':100},Taylor,48441,False,False,America/Chicago
2,Abilene Regional Medical Center,3,6250 US HIGHWAY 83,NaN,ABILENE,TX,79606,Taylor,79606.0,32.36095,...,25290.0,141.4,48441.0,Taylor,{'48441':100},Taylor,48441,False,False,America/Chicago
3,Abilene Regional Medical Center Skilled Nursin...,4,6250 HIGHWAY 83 84 AT ANTILLEY RD,NaN,ABILENE,TX,79606,Taylor,79606.0,32.36095,...,25290.0,141.4,48441.0,Taylor,{'48441':100},Taylor,48441,False,False,America/Chicago
4,"Absolutely Angels, Inc",5,936 E PRODUCTION DR,NaN,PILOT POINT,TX,76258,Denton,76258.0,33.37190,...,6876.0,28.8,48121.0,Denton,"{'48121':95.03,'48181':4.97}",Denton|Grayson,48121|48181,False,False,America/Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,Wortham Rehabilitation Center,1809,117 TWIN CIR,NaN,WORTHAM,TX,76693,Freestone,76693.0,31.78599,...,1629.0,8.2,48161.0,Freestone,"{'48161':98.3,'48293':1.7}",Freestone|Limestone,48161|48293,False,False,America/Chicago
1809,Yoakum Community Hospital Swing Bed,1810,1200 CARL RAMERT DR,NaN,YOAKUM,TX,77995,Lavaca,77995.0,29.21180,...,10411.0,10.4,48285.0,Lavaca,"{'48123':40.25,'48285':56.86,'48469':2.88}",Lavaca|DeWitt|Victoria,48285|48123|48469,False,False,America/Chicago
1810,Yoakum County Hospital,1811,412 MUSTANG DR,NaN,DENVER CITY,TX,79323,Yoakum,79323.0,32.94697,...,7432.0,11.0,48501.0,Yoakum,"{'48165':12.01,'48501':87.99}",Yoakum|Gaines,48501|48165,False,False,America/Chicago
1811,Yoakum Nursing and Rehabilitation Center,1812,1300 CARL RAMERT DR,NaN,YOAKUM,TX,77995,Lavaca,77995.0,29.21180,...,10411.0,10.4,48285.0,Lavaca,"{'48123':40.25,'48285':56.86,'48469':2.88}",Lavaca|DeWitt|Victoria,48285|48123|48469,False,False,America/Chicago


In [89]:
# combine two dataset
county_complete = county_complete[["ProviderName","PID","lat","lng","county_name"]]

In [90]:
# combine two dataset to attain a final dataset
result = pd.merge(df_, county_complete, how='left', on='PID')

In [91]:
result.to_csv("Data/curr_complete_dataset_delete_columns.csv",index=False)